In [1]:
#!/usr/bin/env python
# coding: utf-8

import torch
import random
from transformers import HfArgumentParser, Seq2SeqTrainingArguments,EarlyStoppingCallback

import logging

from dataclasses import dataclass, field
from typing import Callable, Dict, Optional
from datasets import load_dataset, concatenate_datasets,Value
import numpy as np
from typing import Union, Optional
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, EvalPrediction, GlueDataset, AutoModel
from transformers import GlueDataTrainingArguments as DataTrainingArguments
from transformers import (
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    #glue_compute_metrics,
    glue_output_modes,
    glue_tasks_num_labels,
    set_seed,GenerationConfig
)
from arguments import ModelArguments, DataArguments
import wandb
from nltk.tokenize import sent_tokenize
import nltk
from evaluate import load

nltk.download("punkt")
logger = logging.getLogger(__name__)

import pathlib
from datasets import load_dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer
from evaluate import load
from utils import *
import numpy as np
from peft import PeftModel    
import logging
import os
from torch.utils.data import DataLoader 
from tqdm import tqdm
from typing import List


from llama import Dialog, Llama
import fire

2024-04-24 22:07:15.199034: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-24 22:07:15.199396: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-24 22:07:15.200403: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-24 22:07:15.207087: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-24 22:07:16.744404: W tensorflow/compiler/tf2

In [19]:
FULL_MODEL_NAME="mistralai/Mistral-7B-v0.1"

tokenizer = AutoTokenizer.from_pretrained(FULL_MODEL_NAME)
val_dataset = get_dataset(
    dataset_path = "boda/kaneko_data",
    split='test',
    field='prompt',
    num_shots=0,
    explicit_errors = 1)



val_dataloader = DataLoader(val_dataset, batch_size=4,collate_fn=lambda x: x )



Map:   0%|          | 0/144 [00:00<?, ? examples/s]

In [2]:


bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            # bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_compute_dtype=torch.bfloat16,
            bnb_4bit_use_double_quant=False
        )
model = AutoModelForCausalLM.from_pretrained(
        FULL_MODEL_NAME,
        quantization_config=bnb_config,
        trust_remote_code=True,
        # use_flash_attention_2=model_args.use_flash_attention_2,
        use_flash_attention_2=0,

        )

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
CHECKPOINT_PATH= '/l/users/abdelrahman.sadallah/UWFE-mistral-explicit-errors/mistralai/Mistral-7B-v0.1/best'


model = PeftModel.from_pretrained(model, CHECKPOINT_PATH)
model = model.eval()


In [37]:
def inference(prompts, tokenizer, model):
    
   
    encoding = tokenizer(prompts, return_tensors="pt", padding=True).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **encoding,
            max_new_tokens=256,
            do_sample=False,
            temperature=0.0001,
            repetition_penalty = 5.0,
            pad_token_id=tokenizer.eos_token_id,
            early_stopping=True,
            num_beams=10,
            # generation_config=generation_config,
        )  
    answer_tokens = outputs[:, encoding.input_ids.shape[1] :]
    output_text = tokenizer.batch_decode(answer_tokens, skip_special_tokens=True)

    return output_text

In [38]:

# Define PAD Token = BOS Token
tokenizer.pad_token = tokenizer.bos_token
model.config.pad_token_id = model.config.bos_token_id

predicitons = []
outputs = []     
for batch in tqdm(val_dataloader):

    prompts = [x['prompt'] for x in batch]
    ans = []
    
    output_text = inference(prompts=prompts, tokenizer=tokenizer, model=model)
    labels = []
    for i,ex in enumerate(batch):
        # labels.append(f"{i+1}. {ex['explanation']}\n")
        labels.append(ex['label'])
    
    predicitons.extend(output_text)
    outputs.extend(labels)

    break



  0%|                                                                                                                                                                  | 0/36 [01:12<?, ?it/s]


In [39]:
for i in range(len(predicitons)):
            print(f"Error_sentence: {val_dataset[i]['incorrect_sentence']}\n")
            print(f"Correct_sentence: {val_dataset[i]['correct_sentence']}\n")
            print(f"Prediction: {predicitons[i]}\n")
            print(f"Label: {outputs[i]}\n")
            print('\n\n\n')

Error_sentence: Therefore , the lack of creative qualified human resource seriously hampers engineering design processes for innovations in China .

Correct_sentence: Therefore , the lack of creative human resources seriously hampers engineering design processes for innovations in China .

Prediction: 
1. The adjective "creative" modifies the noun "human resources," so it doesn't need to be repeated.
2. This sentence is talking about human resources in general, so "resource" should be pluralized. Also, the verb "hampers" is not used with countable nouns, so "resources" should be changed to the uncountable noun "resource." 
3. None

Label: 1. The adjective "qualified" means having the necessary skills to perform a particular task, but it refers mainly to official or certified qualifications, such as tests. There is no test that would certify someone's creative thinking abilities, so "qualified" should be omitted. The sentence could also be reworded to "human resources with creative thin

In [1]:
from utils import get_dataset

ds =  get_dataset(
    dataset_path = "boda/kaneko_data",
    split='test',
    field='prompt',
    num_shots=5,
    explicit_errors = 0).select([0])

2024-04-25 12:02:21.254619: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-25 12:02:21.254877: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-25 12:02:21.302062: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-25 12:02:21.402663: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-25 12:02:23.045192: W tensorflow/compiler/tf2

In [2]:
ds

Dataset({
    features: ['corrections', 'incorrect_sentence', 'correct_sentence', 'prompt', 'label'],
    num_rows: 1
})